In [ ]:
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm

# 1. MUAT MODEL (Sama seperti kode Anda)
print("Memuat model dan tokenizer...")
pretrained= "mdhugol/indonesia-bert-sentiment-classification"
model = AutoModelForSequenceClassification.from_pretrained(pretrained)
tokenizer = AutoTokenizer.from_pretrained(pretrained)

sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
label_index = {'LABEL_0': 'positif', 'LABEL_1': 'netral', 'LABEL_2': 'negatif'}
print("Model berhasil dimuat.")

# 2. TENTUKAN FILE INPUT/OUTPUT DAN NAMA KOLOM
input_csv_path = 'GojekReview.csv'
output_csv_path = 'hasil_sentimen.csv'
text_column_name = 'full_text' # Sesuaikan dengan nama kolom di file CSV Anda

# 3. FUNGSI UNTUK ANALISIS SENTIMEN SATU TEKS
def analyze_sentiment(text):
    """
    Fungsi ini mengambil satu string teks, menjalankannya melalui pipeline
    analisis sentimen, dan mengembalikan label dan skornya.
    """
    # Pastikan input adalah string, jika tidak (misal: float, NaN) kembalikan netral
    if not isinstance(text, str):
        return 'neutral', 0.0

    try:
        result = sentiment_analysis(text)[0]
        label = label_index[result['label']]
        score = result['score']
        return label, score
    except Exception as e:
        print(f"Error saat menganalisis teks: '{text}'. Error: {e}")
        return 'error', 0.0

# 4. PROSES UTAMA
try:
    # Baca file CSV
    print(f"Membaca file CSV dari: {input_csv_path}")
    df = pd.read_csv(input_csv_path, encoding = 'latin1')

    # Menyiapkan tqdm untuk progress bar di pandas
    tqdm.pandas(desc="Menganalisis sentimen")

    # Terapkan fungsi analyze_sentiment ke setiap baris di kolom teks
    # .progress_apply akan menampilkan progress bar
    # Hasilnya akan berupa tuple (label, score)
    results = df[text_column_name].progress_apply(analyze_sentiment)

    # Pisahkan tuple hasil menjadi dua kolom baru: 'sentimen' dan 'skor_sentimen'
    df[['sentimen', 'skor_sentimen']] = pd.DataFrame(results.tolist(), index=df.index)

    # Simpan DataFrame yang sudah diperbarui ke file CSV baru
    print(f"Menyimpan hasil ke: {output_csv_path}")
    df.to_csv(output_csv_path, index=False) # index=False agar tidak ada kolom indeks tambahan

    print("Proses selesai!")

except FileNotFoundError:
    print(f"Error: File '{input_csv_path}' tidak ditemukan. Pastikan file ada di direktori yang sama.")
except KeyError:
    print(f"Error: Kolom '{text_column_name}' tidak ditemukan di dalam file CSV. Periksa kembali nama kolom.")